In [1]:
from ray import tune
import numpy as np
import pandas as pd
import os
import datetime
from ray.tune.suggest.bayesopt import BayesOptSearch
from one_region_simu import *
from evaluation import *
import pdb
import sys

In [2]:
data_folder = os.path.join(os.getcwd(), 'data')
exp_name = '20220606_trial1'
ray_folder = os.path.join(data_folder,'ray_results')
exp_folder = os.path.join(ray_folder, exp_name)
restore_folder = os.path.join(ray_folder,'20220606_trial1')
os.makedirs(exp_folder, exist_ok=True)
# os.environ['RAY_PICKLE_VERBOSE_DEBUG']='1'

In [3]:
config={'ntau':tune.uniform(0.0000001,10),
        # 'nsig': tune.uniform(0.001,0.01),
        "c_ee": tune.uniform(12.5, 13.5),
        "c_ei": tune.uniform(7, 10),
        'c_ie': tune.uniform(6, 18),
        'c_ii': tune.uniform(7, 10),
        'tau_e': tune.uniform(0, 150),
        'tau_i': tune.uniform(0, 150),
        'r_e': tune.uniform(0.5, 2),
        'r_i': tune.uniform(0.5, 2)
        }

def run_model(config):
    params = config_params({"c_ee":config['c_ee'],'c_ei':config['c_ei'],'c_ie':config['c_ie'],'c_ii':config['c_ii'],
                   'tau_e':config['tau_e'],'tau_i':config['tau_i'],'r_e':config['r_e'],'r_i':config['r_i']})
    score = eva_limit_cycle(params)
    if not score:
        region = config_one_region()
        surface = config_surface(region)
        sim = config_simulator(params, region, surface)
        result_name = run_simulation(sim, params, check_point=12500,data_folder=data_folder)
        score = score(result_name)
    tune.report(score=score)

In [4]:
# # Customize the logger
from typing import Dict, List
import json
from ray.tune.logger import LoggerCallback

class CustomLoggerCallback(LoggerCallback):
    """Put the results of all trails in the same file"""

    def __init__(self, filefolder, filename: str = "log.txt"):
        self._filename = filename
        self._filepath = os.path.join(filefolder, self._filename)
        
    def log_trial_start(self, trial: "Trial"):
        self._file = open(self._filepath, "at")
        self._file.write(json.dumps(trial.evaluated_params))

    def log_trial_result(self, iteration: int, trial: "Trial", result: Dict):
        self._file = open(self._filepath, "at")
        try:
            self._file.write(json.dumps(result))
        except TypeError:
            for idx, (k, v) in enumerate(result.items()):
                if isinstance(v, (np.ndarray,)):
                    result[k] = v.tolist()
            self._file.write(json.dumps(result))

    def on_trial_complete(self, iteration: int, trials: List["Trial"],
                          trial: "Trial", **info):
        self._file = open(self._filepath, "at")
        self._file.write(str(datetime.datetime.now())+'\n')
        self._file.close()

In [5]:
smoke_test = False
restore = False
nsample = -1
debug = False
bayesopt = BayesOptSearch(metric="score", mode="min",random_search_steps=5000)
# pdb.set_trace()
if restore:
    bayesopt.restore(os.path.join(restore_folder, 'baye_checkpoint.pkl'))
    # bayesopt._total_random_search_trials=99
if debug:
    sys.path.append('/opt/app-root/src/.local/lib/python3.8/site-packages/ray/tune')
    pdb.set_trace()
# try:
analysis = tune.run(run_model,
                    verbose=3,
                    name = exp_name,
                    local_dir=ray_folder,
                    search_alg=bayesopt,
                    config=config,
                    metric="score",
                    mode="min",
                    progress_reporter=tune.JupyterNotebookReporter(overwrite=True,metric_columns='score',
                                                                  parameter_columns=config),
                    callbacks=[CustomLoggerCallback(filefolder=exp_folder, filename= "log_test.txt")],
                    stop={'score':0.08},
                    # resume=True, # do this when you whant to resume a stopped experiment
                    max_concurrent_trials = 2,
                    num_samples=2 if smoke_test else nsample)
# except ValueError as e:
#     # TODO: Add the error!
#     if str(e) != "blablabla":
#             raise
#     else:
#         analysis = tune.run(run_model,
#                             verbose=3,
#                             name = exp_name,
#                             local_dir=ray_folder,
#                             search_alg=bayesopt,
#                             # config=config,
#                             metric="score",
#                             mode="min",
#                             progress_reporter=tune.JupyterNotebookReporter(overwrite=True,metric_columns='score',
#                                                                           parameter_columns=config),
#                             callbacks=[CustomLoggerCallback(filefolder=exp_folder, filename= "log_test.txt")],
#                             stop={'score':0.08},
#                             # resume=True, # do this when you whant to resume a stopped experiment
#                             max_concurrent_trials = 2,
#                             num_samples=2 if smoke_test else nsample)

Trial name,status,loc,<ray.tune.sample.Float object at 0x7fae2d1b16d0>,<ray.tune.sample.Float object at 0x7fae2d1b1760>,<ray.tune.sample.Float object at 0x7fae2d1b17f0>,<ray.tune.sample.Float object at 0x7fae2d1b1880>,<ray.tune.sample.Float object at 0x7fae2d1b1910>,<ray.tune.sample.Float object at 0x7fae2d1b19d0>,<ray.tune.sample.Float object at 0x7fae2d1b1a90>,<ray.tune.sample.Float object at 0x7fae2d1b1b20>,<ray.tune.sample.Float object at 0x7fae2d1b1bb0>
run_model_6b43228c,RUNNING,10.130.5.143:130049,7.17187,13.0717,9.81336,6.0911,8.71801,0.660241,0.0495239,1.61762,1.85747
run_model_3f15a5f4,RUNNING,10.130.5.143:4228,5.82886,13.0683,9.39438,12.3065,7.2732,114.182,115.616,1.41761,1.59306
run_model_f59bb5a4,TERMINATED,10.130.5.143:67329,1.56019,12.8745,9.85214,14.7839,8.79598,129.926,90.1673,0.733992,0.587125
run_model_f8b1d9b2,TERMINATED,10.130.5.143:67363,2.12339,13.2081,7.06175,17.6389,9.49733,45.6363,78.7135,0.772737,0.775107
run_model_fe629cde,TERMINATED,10.130.5.143:67399,2.92145,12.9319,7.87369,13.3422,7.41848,117.776,29.9511,1.04954,1.1841
run_model_01c49940,TERMINATED,10.130.5.143:67432,1.70524,13.0142,8.77724,6.5574,8.82263,144.845,121.26,0.597577,1.92333
run_model_0747ebb0,TERMINATED,10.130.5.143:67468,1.22038,12.8046,7.29302,14.2108,8.32046,136.398,38.817,1.24277,0.551583
run_model_0a9e02c2,TERMINATED,10.130.5.143:67501,1.84854,13.1625,7.93513,12.2408,8.64013,140.925,134.224,1.95438,1.6627
run_model_100f8a46,TERMINATED,10.130.5.143:67537,0.452273,13.0979,9.76562,7.06191,7.58795,40.7024,124.311,0.987995,1.08302
run_model_13c5752e,TERMINATED,10.130.5.143:67570,8.02197,12.8568,7.8428,12.5124,7.42277,115.837,29.8074,0.611826,1.98033


  ERROR  Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "/srv/main-spack-instance/spack/var/spack/environments/release_v0_1_202109_hotfix/.spack-env/view/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_67055/1855535263.py", line 14, in <module>
    analysis = tune.run(run_model,
  File "/opt/app-root/src/.local/lib/python3.8/site-packages/ray/tune/tune.py", line 686, in run
    runner.cleanup()
  File "/opt/app-root/src/.local/lib/python3.8/site-packages/ray/tune/trial_runner.py", line 1382, in cleanup
    self.cleanup_trials()
  File "/opt/app-root/src/.local/lib/python3.8/site-packages/ray/tune/trial_runner.py", line 1378, in cleanup_trials
    self.trial_executor.cleanup(self.get_trials())
  File "/opt/app-root/src/.local/lib/python3.8/site-packages/ray/tune/ray_trial_executor.py",

TypeError: object of type 'NoneType' has no len()

In [ ]:
# Show the output after a long time
# output.show()

In [ ]:
# save the current experiment
bayesopt.save(os.path.join(exp_folder, 'baye_checkpoint.pkl'))

In [ ]:
print("Best hyperparameters found were: ", analysis.best_config)
print("Best result: ", analysis.best_result)
analysis.results_df

## Load the result from file

In [ ]:
from ray.tune import ExperimentAnalysis
analysis = ExperimentAnalysis('/mnt/user/drive/My Libraries/tutorials&explorations/data/ray_results/20220526_trial1')

In [ ]:
analysis.get_best_config(metric="score",mode="min",)

In [ ]:
analysis.trials[0].metric_analysis

In [ ]:
min(analysis.results_df.score)

(run_model pid=130049) 2022-06-06 13:04:21,291	ERROR function_runner.py:281 -- Runner Thread raised error.
(run_model pid=130049) Traceback (most recent call last):
(run_model pid=130049)   File "/opt/app-root/src/.local/lib/python3.8/site-packages/ray/tune/function_runner.py", line 272, in run
(run_model pid=130049)     self._entrypoint()
(run_model pid=130049)   File "/opt/app-root/src/.local/lib/python3.8/site-packages/ray/tune/function_runner.py", line 348, in entrypoint
(run_model pid=130049)     return self._trainable_func(
(run_model pid=130049)   File "/opt/app-root/src/.local/lib/python3.8/site-packages/ray/util/tracing/tracing_helper.py", line 462, in _resume_span
(run_model pid=130049)     return method(self, *_args, **_kwargs)
(run_model pid=130049)   File "/opt/app-root/src/.local/lib/python3.8/site-packages/ray/tune/function_runner.py", line 640, in _trainable_func
(run_model pid=130049)     output = fn()
(run_model pid=130049)   File "/tmp/ipykernel_67055/2549213022.py",